Plant Parenthood

In [3]:
%reload_ext sql


In [4]:
%sql sqlite:///Plant_Parenthood.db

In [5]:
import pymysql

In [8]:
%%sql
SELECT name
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///Plant_Parenthood.db
Done.


name
CUSTOMERS
PLANTS
Orders
STATUS
ORDER_DETAILS


In [23]:
%%sql
SELECT P.name AS PlantName, OD.quantity, O.OrderDate
FROM PLANTS P
JOIN ORDER_DETAILS OD ON P.pid = OD.pid
JOIN ORDERS O ON OD.oid = O.oid
WHERE O.cid = 1;  

 * sqlite:///Plant_Parenthood.db
Done.


PlantName,quantity,OrderDate


In [20]:
%%sql
SELECT * FROM ORDER_DETAILS


 * sqlite:///Plant_Parenthood.db
Done.


oid,pid,quantity,price
1,1,2,20.0
13,1,2,20.0
432424,1,1,23.0


In [21]:
%%sql
SELECT * FROM CUSTOMERS

 * sqlite:///Plant_Parenthood.db
Done.


cid,name,email
1,dylan,dylana
2,cora,cora@gmail.com
3,comfort,comfort@gmail.com


In [22]:
%%sql
SELECT * FROM PLANTS

 * sqlite:///Plant_Parenthood.db
Done.


pid,name,quantity
1,pothos,2
2,monstera,4


In [24]:
%%sql
SELECT * FROM orders

 * sqlite:///Plant_Parenthood.db
Done.


oid,cid,OrderDate,shippingAddress
1,2,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
2,1,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
3,2,2024-09-09,"500 Chapel Hill Dr, Chapel Hill, NC"
4,3,2024-10-09,"700 Chapel Hill Dr, Chapel Hill, NC"
5,2,today,graham memorial


In [9]:
%%sql
-- Add FOREIGN KEY to link `ORDER_DETAILS` to `ORDERS`
ALTER TABLE ORDER_DETAILS
ADD CONSTRAINT fk_orderdetails_orders
FOREIGN KEY (oid) REFERENCES ORDERS (oid) ON DELETE CASCADE;

-- Add FOREIGN KEY to link `ORDER_DETAILS` to `PLANTS`
ALTER TABLE ORDER_DETAILS
ADD CONSTRAINT fk_orderdetails_plants
FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE;

 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: -- Add FOREIGN KEY to link `ORDER_DETAILS` to `ORDERS`
ALTER TABLE ORDER_DETAILS
ADD CONSTRAINT fk_orderdetails_orders
FOREIGN KEY (oid) REFERENCES ORDERS (oid) ON DELETE CASCADE;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [18]:
%%sql
SELECT * FROM ORDERS

 * sqlite:///Plant_Parenthood.db
Done.


oid,cid,OrderDate,shippingAddress
1,2,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
2,1,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
3,2,2024-09-09,"500 Chapel Hill Dr, Chapel Hill, NC"
4,3,2024-10-09,"700 Chapel Hill Dr, Chapel Hill, NC"
5,2,today,graham memorial


In [7]:
%%sql
CREATE TABLE ORDER_DETAILS (
    oid INTEGER NOT NULL,             -- Order ID
    pid INTEGER NOT NULL,             -- Plant ID
    quantity INTEGER NOT NULL,        -- Quantity of the plant in the order
    price REAL NOT NULL,              -- Price per unit of the plant
    PRIMARY KEY (oid, pid),           -- Composite primary key
    FOREIGN KEY (oid) REFERENCES ORDERS (oid) ON DELETE CASCADE,  -- Links to ORDERS table
    FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE   -- Links to PLANTS table
);

 * sqlite:///Plant_Parenthood.db
Done.


[]

In [62]:
%%sql
CREATE TABLE CUSTOMERS(
    cid INTEGER PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email TEXT);

 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) table CUSTOMERS already exists
[SQL: CREATE TABLE CUSTOMERS(
    cid INTEGER PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email TEXT);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [63]:
%%sql
CREATE TABLE PLANTS(
    pid INTEGER PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    quantity INTEGER
    );

 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) table PLANTS already exists
[SQL: CREATE TABLE PLANTS(
    pid INTEGER PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    quantity INTEGER
    );]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [64]:
%%sql
CREATE TABLE Orders (
    oid INTEGER PRIMARY KEY,
    cid INTEGER NOT NULL,
    OrderDate TEXT NOT NULL,
    shippingAddress TEXT NOT NULL,
    FOREIGN KEY (cid) REFERENCES CUSTOMERS (cid)
);


 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) table Orders already exists
[SQL: CREATE TABLE Orders (
    oid INTEGER PRIMARY KEY,
    cid INTEGER NOT NULL,
    OrderDate TEXT NOT NULL,
    shippingAddress TEXT NOT NULL,
    FOREIGN KEY (cid) REFERENCES CUSTOMERS (cid)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [65]:
%%sql
CREATE TABLE STATUS (
    cid INTEGER,
    oid INTEGER,
    status TEXT CHECK (status IN ('success', 'pending')),
    PRIMARY KEY (cid, oid),
    FOREIGN KEY (cid) REFERENCES CUSTOMERS (cid) ON DELETE CASCADE,
    FOREIGN KEY (oid) REFERENCES Orders (oid) ON DELETE CASCADE
);

 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) table STATUS already exists
[SQL: CREATE TABLE STATUS (
    cid INTEGER,
    oid INTEGER,
    status TEXT CHECK (status IN ('success', 'pending')),
    PRIMARY KEY (cid, oid),
    FOREIGN KEY (cid) REFERENCES CUSTOMERS (cid) ON DELETE CASCADE,
    FOREIGN KEY (oid) REFERENCES Orders (oid) ON DELETE CASCADE
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
%%sql
CREATE TRIGGER UpdateStatusOnInsert
AFTER INSERT ON STATUS
FOR EACH ROW
BEGIN
    -- If plant quantity is sufficient, set status to 'success'
    UPDATE STATUS
    SET status = 'success'
    WHERE cid = NEW.cid
    AND oid = NEW.oid
    AND (SELECT quantity FROM PLANTS WHERE PLANTS.pid = NEW.oid AND PLANTS.quantity > 0) IS NOT NULL;

    -- If plant quantity is insufficient, set status to 'pending'
    UPDATE STATUS
    SET status = 'pending'
    WHERE cid = NEW.cid
    AND oid = NEW.oid
    AND (SELECT quantity FROM PLANTS WHERE PLANTS.pid = NEW.oid AND PLANTS.quantity <= 0) IS NOT NULL;
END;


 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) trigger UpdateStatusOnInsert already exists
[SQL: CREATE TRIGGER UpdateStatusOnInsert
AFTER INSERT ON STATUS
FOR EACH ROW
BEGIN
    -- If plant quantity is sufficient, set status to 'success'
    UPDATE STATUS
    SET status = 'success'
    WHERE cid = NEW.cid
    AND oid = NEW.oid
    AND (SELECT quantity FROM PLANTS WHERE PLANTS.pid = NEW.oid AND PLANTS.quantity > 0) IS NOT NULL;

    -- If plant quantity is insufficient, set status to 'pending'
    UPDATE STATUS
    SET status = 'pending'
    WHERE cid = NEW.cid
    AND oid = NEW.oid
    AND (SELECT quantity FROM PLANTS WHERE PLANTS.pid = NEW.oid AND PLANTS.quantity <= 0) IS NOT NULL;
END;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [11]:
%%sql
CREATE TRIGGER AddToStatusAfterOrderInsert
AFTER INSERT ON Orders
FOR EACH ROW
BEGIN
    INSERT INTO STATUS (cid, oid, status)
    VALUES (NEW.cid, NEW.oid, 'pending');
END;


 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) trigger AddToStatusAfterOrderInsert already exists
[SQL: CREATE TRIGGER AddToStatusAfterOrderInsert
AFTER INSERT ON Orders
FOR EACH ROW
BEGIN
    INSERT INTO STATUS (cid, oid, status)
    VALUES (NEW.cid, NEW.oid, 'pending');
END;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [12]:
%%sql
CREATE TRIGGER UpdatePlantQuantityAfterOrder
AFTER INSERT ON ORDER_DETAILS
FOR EACH ROW
BEGIN
    UPDATE PLANTS
    SET quantity = quantity - NEW.quantity
    WHERE pid = NEW.pid;
END;

 * sqlite:///Plant_Parenthood.db
Done.


[]

In [86]:
%%sql
CREATE TABLE ORDERDETAILS (
    oid INTEGER NOT NULL,
    pid INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES Orders (oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE
);


 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) table ORDERDETAILS already exists
[SQL: CREATE TABLE ORDERDETAILS (
    oid INTEGER NOT NULL,
    pid INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES Orders (oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [69]:
%%sql
select * from orders

 * sqlite:///Plant_Parenthood.db
Done.


oid,cid,OrderDate,shippingAddress
1,1,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
2,1,2024-08-09,"100 Chapel Hill Dr, Chapel Hill, NC"
3,2,2024-09-09,"500 Chapel Hill Dr, Chapel Hill, NC"
4,3,2024-10-09,"700 Chapel Hill Dr, Chapel Hill, NC"
11111,222,3,3


In [70]:
%%sql
select * from customers

 * sqlite:///Plant_Parenthood.db
Done.


cid,name,email
1,dylan,dylan@gmail.com
2,cora,cora@gmail.com
3,comfort,comfort@gmail.com


In [71]:
%%sql
select * from status

 * sqlite:///Plant_Parenthood.db
Done.


cid,oid,status
1,2,pending
2,3,pending
3,4,pending


In [72]:
%%sql
insert into customers(cid, name, email)
values(3, 'comfort', 'comfort@gmail.com')

 * sqlite:///Plant_Parenthood.db
(sqlite3.IntegrityError) UNIQUE constraint failed: CUSTOMERS.cid
[SQL: insert into customers(cid, name, email)
values(3, 'comfort', 'comfort@gmail.com')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [73]:
%%sql
INSERT INTO ORDERS (oid, cid, orderdate, shippingAddress)
VALUES (4, 3, '2024-10-09', '700 Chapel Hill Dr, Chapel Hill, NC');


 * sqlite:///Plant_Parenthood.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Orders.oid
[SQL: INSERT INTO ORDERS (oid, cid, orderdate, shippingAddress)
VALUES (4, 3, '2024-10-09', '700 Chapel Hill Dr, Chapel Hill, NC');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [74]:
%%sql
CREATE TABLE IF NOT EXISTS ORDER_DETAILS (
    oid INTEGER NOT NULL,
    pid INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES ORDERS (oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE
);

 * sqlite:///Plant_Parenthood.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: CREATE TABLE IF NOT EXISTS ORDER_DETAILS (
    oid INTEGER NOT NULL,
    pid INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES ORDERS (oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES PLANTS (pid) ON DELETE CASCADE
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [75]:
%%sql
select * from plants

 * sqlite:///Plant_Parenthood.db
Done.


pid,name,quantity


In [76]:
def validate_column_value(column_name, column_type, not_null, value):
    """
    Validates a single column value based on its type and constraints.

    Parameters:
    - column_name (str): The name of the column.
    - column_type (str): The expected type of the column (e.g., INTEGER, REAL, TEXT, BOOLEAN).
    - not_null (bool): Whether the column has a NOT NULL constraint.
    - value (str): The user-provided value to validate.

    Returns:
    - str/int/float: The validated and properly formatted value if valid.
    - None: If validation fails (prints an error and asks for re-entry).
    """
    # Normalize column_type for compatibility with SQLite types
    normalized_type = column_type.split('(')[0].upper()  # Remove size specifiers (e.g., VARCHAR(255) -> VARCHAR)

    # Map normalized types to base SQLite types
    if normalized_type in ["VARCHAR", "CHAR", "CLOB"]:
        normalized_type = "TEXT"
    elif normalized_type in ["FLOAT", "DOUBLE", "DECIMAL"]:
        normalized_type = "REAL"
    elif normalized_type in ["INT", "TINYINT", "SMALLINT", "BIGINT"]:
        normalized_type = "INTEGER"

    # NOT NULL validation
    if not_null and not value.strip():
        print(f"Error: {column_name} cannot be null.")
        return None

    # Type validation
    if normalized_type == "INTEGER":
        if not value.isdigit():
            print(f"Error: {column_name} must be an integer.")
            return None
        return int(value)  # Convert to integer

    elif normalized_type == "REAL":
        try:
            return float(value)  # Convert to float
        except ValueError:
            print(f"Error: {column_name} must be a real number.")
            return None

    elif normalized_type == "TEXT":
        if not isinstance(value, str):
            print(f"Error: {column_name} must be text.")
            return None
        return value.strip()  # Return as string

    elif normalized_type == "BOOLEAN":
        if value.lower() in ["true", "1"]:
            return 1  # SQLite uses 1 for true
        elif value.lower() in ["false", "0"]:
            return 0  # SQLite uses 0 for false
        else:
            print(f"Error: {column_name} must be a boolean (true/false or 1/0).")
            return None

    else:
        print(f"Error: Unknown column type '{column_type}' for {column_name}.")
        return None


In [78]:
import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('Plant_Parenthood.db')
cursor = conn.cursor()

# Fetch the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

# Check if tables exist in the database
if not tables:
    print("No tables found in the database. Please create a table before proceeding.")
    conn.close()
    exit()

def display_menu():
    # Display available tables and let the user choose one
    print("\nAvailable tables:")
    for i, table in enumerate(tables, start=1):
        print(f"{i}. {table}")

    try:
        table_choice = int(input("Select the table you want to alter (by number): ")) - 1
        if table_choice < 0 or table_choice >= len(tables):
            raise ValueError("Invalid table choice.")
        return tables[table_choice]
    except ValueError:
        print("Invalid choice. Exiting.")
        conn.close()
        exit()

def perform_action(selected_table):
    # Menu for selecting an action
    print("\nChoose an action:")
    print("1. Add a new record")
    print("2. Delete a record")
    print("3. View all records")
    print("4. Update a record")
    print("5. View plants ordered by Customer ID")
    choice = input("Enter your choice: ")

    if choice == "1":
        # Add a new record
        print("\nAdding a new record:")
        columns_query = f"PRAGMA table_info({selected_table});"
        cursor.execute(columns_query)
        columns_info = cursor.fetchall()  # Fetch column details: (cid, name, type, not null, etc.)

        values = []
        for column_info in columns_info:
            # Correctly map each field from the PRAGMA output
            column_name = column_info[1]  # Column name
            column_type = column_info[2]  # Data type (e.g., INTEGER, TEXT)
            not_null = column_info[3]     # NOT NULL constraint (1 or 0)
            default_value = column_info[4]  # Default value
            primary_key = column_info[5]  # Primary key flag (1 or 0)

            # Prompt and validate input
            while True:
                value = input(f"Enter value for {column_name} ({column_type}): ")
                validated_value = validate_column_value(column_name, column_type, not_null, value)
                if validated_value is not None:  # If validation passes, add the value
                    values.append(validated_value)
                    break

        # Prepare and execute the INSERT query
        columns_names = ", ".join([col[1] for col in columns_info])
        placeholders = ", ".join(["?"] * len(columns_info))
        query = f"INSERT INTO {selected_table} ({columns_names}) VALUES ({placeholders})"

        try:
            cursor.execute(query, values)
            conn.commit()
            print("New record added successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: {e}")

    elif choice == "2":
        # Delete a record
        delete_column = input(f"Enter the column name to filter by for deletion: ")
        delete_value = input(f"Enter the value for {delete_column} to delete: ")
        
        try:
            cursor.execute(f"DELETE FROM {selected_table} WHERE {delete_column} = ?", (delete_value,))
            conn.commit()
            print("Record deleted successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "3":
        # View all records
        print("\nViewing all records:")
        try:
            cursor.execute(f"SELECT * FROM {selected_table}")
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "4":
        # Update a record
        print("\nUpdating a record:")
        update_column = input("Enter the column name you want to update (e.g., Email): ")
        new_value = input(f"Enter the new value for {update_column}: ")
        filter_column = input("Enter the column name to filter by (e.g., CustomerID): ")
        filter_value = input(f"Enter the value of {filter_column} to identify the record: ")
        
        try:
            cursor.execute(
                f"UPDATE {selected_table} SET {update_column} = ? WHERE {filter_column} = ?",
                (new_value, filter_value)
            )
            conn.commit()
            print("Record updated successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "5":
        #View Orders by Customer ID
        print("\nFinding plants ordered by customer ID:")
        find_cid = input("Enter customer ID (e.g., 3):")

        try:
            procedure_sql = """
            CREATE PROCEDURE find_orders(IN curr_cid INT)
            BEGIN
            SELECT P.name FROM PLANTS P, Orders O WHERE curr_cid = O.cid AND O.oid = P.pid
            END """
            
            cursor.callproc('find_orders', (find_cid))
            rows = cursor.fetchall()
            for row in rows:
                print(row)
         except sqlite3.OperationalError as e:
            print(f"Error: {e}")
            

    else:
        print("Invalid choice. Returning to main menu.")

while True:
    selected_table = display_menu()
    perform_action(selected_table)

    # Ask the user if they want to perform another action
    continue_choice = input("\nDo you want to perform another action? (yes/no): ").strip().lower()
    if continue_choice != "yes":
        print("Exiting program. Goodbye!")
        break

# Close the connection
conn.close()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 136)

In [ ]:
import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('Plant_Parenthood.db')
cursor = conn.cursor()

# Fetch the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

# Check if tables exist in the database
if not tables:
    print("No tables found in the database. Please create a table before proceeding.")
    conn.close()
    exit()

def display_menu():
    # Display available tables and let the user choose one
    print("\nAvailable tables:")
    for i, table in enumerate(tables, start=1):
        print(f"{i}. {table}")

    try:
        table_choice = int(input("Select the table you want to alter (by number): ")) - 1
        if table_choice < 0 or table_choice >= len(tables):
            raise ValueError("Invalid table choice.")
        return tables[table_choice]
    except ValueError:
        print("Invalid choice. Exiting.")
        conn.close()
        exit()

def perform_action(selected_table):
    # Menu for selecting an action
    print("\nChoose an action:")
    print("1. Add a new record")
    print("2. Delete a record")
    print("3. View all records")
    print("4. Update a record")
    print("5. View plants ordered by Customer ID")
    choice = input("Enter your choice: ")

    if choice == "1":
        # Add a new record
        print("\nAdding a new record:")
        columns_query = f"PRAGMA table_info({selected_table});"
        cursor.execute(columns_query)
        columns_info = cursor.fetchall()  # Fetch column details: (cid, name, type, not null, etc.)

        values = []
        for column_info in columns_info:
            # Correctly map each field from the PRAGMA output
            column_name = column_info[1]  # Column name
            column_type = column_info[2]  # Data type (e.g., INTEGER, TEXT)
            not_null = column_info[3]     # NOT NULL constraint (1 or 0)
            default_value = column_info[4]  # Default value
            primary_key = column_info[5]  # Primary key flag (1 or 0)

            # Prompt and validate input
            while True:
                value = input(f"Enter value for {column_name} ({column_type}): ")
                validated_value = validate_column_value(column_name, column_type, not_null, value)
                if validated_value is not None:  # If validation passes, add the value
                    values.append(validated_value)
                    break

        # Prepare and execute the INSERT query
        columns_names = ", ".join([col[1] for col in columns_info])
        placeholders = ", ".join(["?"] * len(columns_info))
        query = f"INSERT INTO {selected_table} ({columns_names}) VALUES ({placeholders})"

        try:
            cursor.execute(query, values)
            conn.commit()
            print("New record added successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: {e}")

    elif choice == "2":
        # Delete a record
        print("\nDeleting a record:")
        try:
            # Display all records to the user
            cursor.execute(f"SELECT * FROM {selected_table}")
            rows = cursor.fetchall()
            
            if not rows:
                print("No records found in the table.")
                return

            # Display rows with indices
            print("\nAvailable records:")
            for row in rows:
                print(row)

            # Assume the first column is the primary key (change if needed)
            primary_key_column = cursor.execute(f"PRAGMA table_info({selected_table})").fetchall()[0][1]

            # Prompt user to select a row based on primary key
            row_id = input(f"\nEnter the {primary_key_column} of the row you want to delete: ")

            # Execute deletion
            cursor.execute(f"DELETE FROM {selected_table} WHERE {primary_key_column} = ?", (row_id,))
            conn.commit()
            print("Record deleted successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")


    elif choice == "3":
        # View all records
        print("\nViewing all records:")
        try:
            cursor.execute(f"SELECT * FROM {selected_table}")
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "4":
        # Update a record
        print("\nUpdating a record:")
        update_column = input("Enter the column name you want to update (e.g., Email): ")
        new_value = input(f"Enter the new value for {update_column}: ")
        filter_column = input("Enter the column name to filter by (e.g., CustomerID): ")
        filter_value = input(f"Enter the value of {filter_column} to identify the record: ")
        
        try:
            cursor.execute(
                f"UPDATE {selected_table} SET {update_column} = ? WHERE {filter_column} = ?",
                (new_value, filter_value)
            )
            conn.commit()
            print("Record updated successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "5":
        #View Orders by Customer ID
        print("\nFinding plants ordered by customer ID:")
        find_cid = input("Enter customer ID (e.g., 3):")

        try:
            procedure_sql = """
            CREATE PROCEDURE find_orders(IN curr_cid INT)
            BEGIN
            SELECT P.name FROM PLANTS P, Orders O WHERE curr_cid = O.cid AND O.oid = P.pid
            END """
            
            cursor.callproc('find_orders', (find_cid))
            rows = cursor.fetchall()
            for row in rows:
                print(row)
         except sqlite3.OperationalError as e:
            print(f"Error: {e}")
            
    else:
        print("Invalid choice. Returning to main menu.")

while True:
    selected_table = display_menu()
    perform_action(selected_table)

    # Ask the user if they want to perform another action
    continue_choice = input("\nDo you want to perform another action? (yes/no): ").strip().lower()
    if continue_choice != "yes":
        print("Exiting program. Goodbye!")
        break

# Close the connection
conn.close()


In [ ]:
%%sql
select * From Orders


In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';